In [ ]:
import pandas as pd
import socket
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import pickle


In [3]:
model = LinearRegression()
test = pd.read_csv('D:\College\Sem2\BigData_4213\data\\test.csv')


<>:2: SyntaxWarning: invalid escape sequence '\C'
<>:2: SyntaxWarning: invalid escape sequence '\C'
C:\Users\DEBJIT\AppData\Local\Temp\ipykernel_6840\31803263.py:2: SyntaxWarning: invalid escape sequence '\C'
  test = pd.read_csv('D:\College\Sem2\BigData_4213\data\\test.csv')


In [ ]:
##


HOST = '127.0.0.1'
PORT = 55872

# Create a TCP socket
server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
server_socket.bind((HOST, PORT))
server_socket.listen(1)

print("Server is listening...")

conn, addr = server_socket.accept()
print(f"Connected by: {addr}")



df_list = []  # Store received chunks

try:
    while True:
        size_prefix = conn.recv(4)  # Read the first 4 bytes (data size)
        if not size_prefix:
            break
        
        data_size = int.from_bytes(size_prefix, 'big')  # Convert to integer
        data = b""
        while len(data) < data_size:
            chunk = conn.recv(data_size - len(data))  # Read full data
            if not chunk:
                break
            data += chunk

        # Deserialize and append to DataFrame list
        received_chunk = pickle.loads(data)
        df_list.append(pd.DataFrame(received_chunk))

        # Combine all chunks
        df = pd.concat(df_list, ignore_index=True)
        df.columns = ['key', 'fare_amount', 'pickup_datetime', 'pickup_longitude', 'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude', 'passenger_count']
        print("Received DataFrame:\n", df.head())
        df.dropna()
        #df.fillna(0,inplace=True)
        
        features = ['pickup_longitude', 'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude', 'passenger_count']
        target = 'fare_amount'
        X = df[features]
        y = df[target]
    
        X = X.dropna()
        y = y.loc[X.index] 
        X_test  = test[features]
        X_test = X_test.dropna()

    
        model.fit(X, y)


        y_pred = model.predict(X_test)

        #start=0
        #batch_size= len(y)
        #while start < len(data):
        #    end = min(start + batch_size, len(data))
        
        m = min(len(y),len(y_pred))
        mse = mean_squared_error(y[:m], y_pred[:m])
        print(f"Mean Squared Error: {mse}")
        
    
    
        print(f"Coefficients: {model.coef_}")
        print(f"Intercept: {model.intercept_}")
    
    # Here we can again send the final model weights and intersept back to the client
    #---TO BE DONE---

except KeyboardInterrupt:
    print("\nServer Stopped by user.")

finally:
    conn.close()
    server_socket.close()
    print("Connection Closed.")


Server is listening...
Connected by: ('127.0.0.1', 53938)
Received DataFrame:
                              key fare_amount          pickup_datetime  \
0    2009-06-15 17:26:21.0000001         4.5  2009-06-15 17:26:21 UTC   
1    2010-01-05 16:52:16.0000002        16.9  2010-01-05 16:52:16 UTC   
2   2011-08-18 00:35:00.00000049         5.7  2011-08-18 00:35:00 UTC   
3    2012-04-21 04:30:42.0000001         7.7  2012-04-21 04:30:42 UTC   
4  2010-03-09 07:51:00.000000135         5.3  2010-03-09 07:51:00 UTC   

  pickup_longitude pickup_latitude dropoff_longitude dropoff_latitude  \
0       -73.844311       40.721319         -73.84161        40.712278   
1       -74.016048       40.711303        -73.979268        40.782004   
2       -73.982738        40.76127        -73.991242        40.750562   
3        -73.98713       40.733143        -73.991567        40.758092   
4       -73.968095       40.768008        -73.956655        40.783762   

  passenger_count  
0               1  
1  

In [24]:
model.predict(X_test)

array([11.25004915, 11.2512942 , 11.25036513, ..., 11.87020422,
       11.87575813, 11.8744723 ], shape=(9914,))